# NFL

https://www.spreadsheet-sports.com/free-tools/2013-nfl-play-play-data-excel/

In [74]:
import numpy as np
import pandas as pd
from __future__ import division
from datetime import datetime

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [102]:
dat = pd.read_excel('data/2014 NFL Play-by-Play Data.xlsx')
dat.head()

,Date,Tm,Opp,Quarter,Time,Down,ToGo,Side of Field,Yard Marker,Tm Score,...,Fantasy Points Rushing Yards,Fantasy Points Receiving Yards,Fantasy Points Receptions,Fantasy Points Rushing TD,Fantasy Points Receiving TD,Fantasy Points Playmaker Fumble,Fantasy Points Passer Yards,Fantasy Points Passer TD,Fantasy Points Passer Interception,Team Game #
0,2014-09-08,Cardinals,Chargers,1,08:32:00,1,14,CRD,4,0,...,0.0,6.3,0.5,0,0,0,2.52,0,0,1
1,2014-09-07,Buccaneers,Panthers,1,03:39:00,1,10,TAM,4,0,...,5.4,0.0,0.0,0,0,0,0.00,0,0,1
2,2014-09-07,Browns,Steelers,1,08:19:00,1,10,CLE,36,0,...,0.0,4.7,0.5,0,0,0,1.88,0,0,1
3,2014-09-07,Jaguars,Eagles,1,01:27:00,1,10,JAX,1,14,...,0.0,4.6,0.5,0,0,0,1.84,0,0,1
4,2014-09-07,Bears,Bills,1,12:11:00,1,10,CHI,44,0,...,0.0,4.4,0.5,0,0,0,1.76,0,0,1


In [139]:
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)

    for col, col_data in X.iteritems():
        teams = pd.get_dummies(col_data, prefix = col)
        outX = outX.join(teams)
    return outX

new_dat = preprocess_features(dat[['Tm', 'Opp']])

In [104]:
# Separate Minute and Second into separate columns - pandas read it in as hour minute so it looks weird 
dat['Minute'] = dat.Time.map(lambda x: x.hour)
dat['Second'] = dat.Time.map(lambda x: x.minute)

In [177]:
# Create new column for what I am trying to predict
def results(play):
    play_type, turnover = play
    if turnover == 1:
        return 'Turnover'
    else:
        return play_type
    
dat['Play Result'] = dat[['Play Type', 'Turnover']].apply(results, axis = 1)

In [179]:
# Create My X and y for classification
X = dat[['Quarter', 'Minute', 'Second', 'Down', 'ToGo', 'Yard Line','Tm Score', 'Opp Score']]
X = X.join(new_dat)
X = X.values
y = dat['Play Result'].values

In [180]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit_transform(y)
le.classes_

array([u'Pass', u'Run', u'Sack', 'Turnover'], dtype=object)

In [181]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 100)

In [182]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

clf = OneVsRestClassifier(SVC())

clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
          n_jobs=1)

In [183]:
clf.score(X_test, y_test)

0.5950413223140496

In [ ]:
Pass, complete, incomplete, run, sack, TD, interception, fumble,